In [12]:
import pandas as pd
import csv
import pickle

from sklearn.neighbors import KNeighborsClassifier  
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


In [ ]:
# Same as prior tests, we gather and clean the data so it is ready to be used in our KNN Model

In [2]:
def cleanBazaarData():
    with open("bazaarData.txt", 'r') as file:
        reader = csv.reader(file, delimiter=';')
        dataset = {}
        n = 0



        for line in reader:
            di = eval(line[1])
            di.update({'product': line[0]})

            reliability = 0
            volumePerc = (di['sellMovingWeek'] / (
                        di['buyMovingWeek'] + di['sellMovingWeek'])) >= .40
            boolWeekly = di['sellMovingWeek'] >= 150000 and di['buyMovingWeek'] >= 150000
            boolOrder = di['sellOrders'] >= 40 and di['buyOrders'] >= 40
            # Determine Weights (A lot of trial and error)

            if di['RoI'] >= 5:
                reliability += 3
                if di['margin'] >= 40:
                    reliability += 3
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                elif 15 < di['margin'] < 40:
                    reliability += 2
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2

            elif 0 < di['RoI'] < 5:
                reliability += 1
                if di['margin'] >= 40:
                    reliability += 3
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                elif 15 < di['margin'] < 40:
                    reliability += 2
                    if volumePerc:
                        reliability += 2
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2
                    else:
                        reliability += 1
                        if boolOrder and boolWeekly:
                            reliability += 1
                        else:
                            reliability -= 2

            else:
                # print(product)
                continue
            di['reliability'] = reliability
            dataset.update({n: di})
            n += 1

        for k in dataset:
            if dataset[k]['reliability'] >= 6:
                dataset[k]['reliability'] = 1
            else:
                dataset[k]['reliability'] = 0




        df = pd.DataFrame.from_dict(dataset,orient='index', columns=['product','buyVolume', 'buyOrders', 'buyPrice', 'buyMovingWeek', 'sellVolume',
                                                      'sellOrders', 'sellPrice', 'sellMovingWeek', 'RoI', 'margin','buySellDiff','reliability'])

        cleanDF = df.drop(columns=['product'])

        

        return cleanDF.drop_duplicates()


data = cleanBazaarData()


In [ ]:
# We drop our feature we want to predict and store it in Y, and create a train test split of our dataset
# We then create our KNN Model and fit it with our Data, predict view the predictions and tweak as we see fit.
# knn is tricky, P is our Power involved with the minkowski formula, weights being distance performed better.
# Leaf Size was fairly irrelevant for this, N_Neighbors performed best between 20-30. Algorithm set to auto I didn't test each.
# Auto will attempt to determine the best algorithm for our model.

In [13]:
y = data['reliability']
X = data.drop(columns=['reliability'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=40)


KNNclass = KNeighborsClassifier(n_neighbors=25,weights='distance',
                                algorithm='auto',leaf_size=30,
                                p=4, metric='minkowski')
KNNclass.fit(X_train, y_train)
pred = KNNclass.predict(X_test)

KNNclassCFM = confusion_matrix(y_test, pred)
print(round((KNNclassCFM[0][0]/(KNNclassCFM[0][0]+KNNclassCFM[1][0]))*100, 2), '% Correct Predictions. KNN Classifier')



97.58 % Correct Predictions. KNN Classifier
